<a href="https://colab.research.google.com/github/Capibara-LLM/Capibara-LLM/blob/main/gemma_2_9b_it_SimPO_Jopara.ipynb"
    target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Basado en https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma2_(9B)-Alpaca.ipynb

### Instalación

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
     import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
     xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
     !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
     !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
     !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None  # None for auto detection
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="princeton-nlp/gemma-2-9b-it-SimPO",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

¡Ahora agregamos adaptadores LoRA para que solo necesitemos actualizar entre el 1 y el 10 % de todos los parámetros!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,              # Rango de la matriz (16 está bien para eficiencia/calidad)
    target_modules = [   # Apuntamos a todos los módulos lineales (Correcto para Gemma 2)
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 32,     # CAMBIO: Aumentado de 16 a 32 (Regla: alpha = 2 * r)
    lora_dropout = 0,    # 0 es correcto para entrenamientos optimizados
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

<a name="Data"></a>
### Data Prep
Carga de datasets: Rubuntu (Filtrado) + Capibara (Concatenado)

In [ ]:
from datasets import load_dataset, concatenate_datasets

# alpaca_prompt = """Iguýpe oĩ peteĩ instrucción omombeꞌuva peteĩ tembiapo, oñembojoajúva peteĩ entrada ndive omeꞌevéva contexto. Ehai peteĩ ñembohovái omohu’ãva hekopete pe mba’ejerure.

# ### Instrucción:
# {}

# ### Entrada:
# {}

# ### Mbohovái:
# {}"""

# EOS_TOKEN = tokenizer.eos_token

#def formatting_prompts_func(examples):
#    instructions = examples["instruction"]
#    inputs       = examples["input"]
#    outputs      = examples["output"]
#    texts = []
#    for instruction, input, output in zip(instructions, inputs, outputs):
#        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#        texts.append(text)
#    return { "text" : texts, }

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    # Plantilla completa (cuando hay contexto/input)
    prompt_with_input = """Iguýpe oĩ peteĩ instrucción omombeꞌuva peteĩ tembiapo, oñembojoajúva peteĩ entrada ndive omeꞌevéva contexto. Ehai peteĩ ñembohovái omohu’ãva hekopete pe mba’ejerure.

### Instrucción:
{}

### Entrada:
{}

### Mbohovái:
{}"""

    # Plantilla simplificada (cuando NO hay input)
    # Nota: Quitamos la intro larga en guaraní para instrucciones simples
    prompt_no_input = """### Instrucción:
{}

### Mbohovái:
{}"""

    EOS_TOKEN = tokenizer.eos_token # Aseguramos que esté disponible

    for instruction, input, output in zip(instructions, inputs, outputs):
        # Si input es None o una cadena vacía/espacios
        if input is None or str(input).strip() == "":
            text = prompt_no_input.format(instruction, output) + EOS_TOKEN
        else:
            text = prompt_with_input.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    return { "text" : texts, }

# --- CARGA Y PROCESAMIENTO DE DATASETS ---

# 1. Cargar Dataset Principal: Rubuntu Guarani-Jopara
print("Cargando dataset Rubuntu...")
dataset_main = load_dataset("rubuntu/dataset-guarani-jopara-v01", split="train")

# FILTRO: Mantener solo filas donde output NO es None y NO está vacío
dataset_main = dataset_main.filter(lambda x: x['output'] is not None and len(str(x['output']).strip()) > 0)
print(f"Dataset Rubuntu filtrado: {len(dataset_main)} filas.")

# 2. Cargar Dataset Secundario: Capibara LLM
print("Cargando dataset Capibara...")
dataset_secondary = load_dataset("Capibara-LLM/gn-multi-affective-alpaca", split="train")
print(f"Dataset Capibara cargado: {len(dataset_secondary)} filas.")

# 3. Estandarización de columnas
def standardize_columns(ds):
    if 'input' not in ds.column_names:
        ds = ds.add_column("input", [""] * len(ds))
    return ds.select_columns(["instruction", "input", "output"])

dataset_main = standardize_columns(dataset_main)
dataset_secondary = standardize_columns(dataset_secondary)

# 4. Concatenar y mezclar
dataset = concatenate_datasets([dataset_main, dataset_secondary])
dataset = dataset.shuffle(seed=3407)
print(f"Total combinado para entrenamiento: {len(dataset)} filas.")

# Aplicar formato
dataset = dataset.map(formatting_prompts_func, batched = True)

<a name="Train"></a>
### Entrenar el modelo

In [ ]:
from trl import SFTConfig, SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Usa 2 núcleos para procesar datos más rápido
    packing = True,       # ¡ACTIVADO! Mucho más rápido y eficiente
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8, # Batch efectivo = 16 (más estable)
        warmup_steps = 10,               # Un poco más de calentamiento para el entrenamiento completo
        # max_steps = 60,                # Entrenar todo el dataset
        num_train_epochs = 1,            # 1 pasada completa a los datos (ajusta a 3 si tienes pocos datos)
        learning_rate = 2e-4,            # 2e-4 está bien con LoRA alpha=32
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

In [ ]:
trainer_stats = trainer.train()

<a name="Inference"></a>
### Inferencia

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Mba'éichapa reiko?", # instruction
        "", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
print(tokenizer.batch_decode(outputs))

<a name="Save"></a>
### Guardar y cargar modelos ajustados
Para guardar el modelo final como adaptadores LoRA, utilizar `push_to_hub` de Huggingface para guardar en línea o `save_pretrained` para guardar localmente.

In [ ]:
from google.colab import userdata

# Define el nombre del modelo
model_name_local = "gemma-2-9b-it-SimPO-Jopara"
repo_id_hub = "Capibara-LLM/gemma-2-9b-it-SimPO-Jopara"

# Obtener el token desde los secretos de Colab
try:
    hf_token = userdata.get('HF_TOKEN')
    if hf_token is None:
        print("Advertencia: El secreto 'HF_TOKEN' no fue encontrado. Usando valor por defecto.")
        hf_token = "TU_TOKEN_HUGGINGFACE_AQUI"
except Exception as e:
    print(f"No se pudo cargar el token de secretos: {e}")
    hf_token = "TU_TOKEN_HUGGINGFACE_AQUI"

print(f"Guardando localmente en: {model_name_local}")
model.save_pretrained(model_name_local)
tokenizer.save_pretrained(model_name_local)

print(f"Subiendo a Hugging Face Hub: {repo_id_hub}")
# Subir automáticamente si tienes el token configurado
if hf_token != "TU_TOKEN_HUGGINGFACE_AQUI":
    model.push_to_hub(repo_id_hub, token = hf_token)
    tokenizer.push_to_hub(repo_id_hub, token = hf_token)
else:
    print("Token no configurado correctamente, saltando subida al Hub.")

### GGUF / llama.cpp Conversion
Opcional: Si se quiere guardar o subir versiones GGUF, cambiar False a True abajo

In [ ]:
if False:
    # RECOMENDADO: Usar q5_k_m para reducir alucinaciones manteniendo un tamaño razonable
    quant_method = "q5_k_m"

    print(f"Guardando y subiendo GGUF con método: {quant_method}")

    # Guardar GGUF localmente
    model.save_pretrained_gguf(model_name_local, tokenizer, quantization_method=quant_method)

    # Subir GGUF al repositorio
    model.push_to_hub_gguf(repo_id_hub + "-GGUF", tokenizer, quantization_method=quant_method, token=hf_token)

if True:
    # Subir múltiples formatos GGUF (q4_k_m, q8_0, q5_k_m)
    model.push_to_hub_gguf(
        repo_id_hub + "-GGUF",
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = hf_token,
    )